**Biomedical Software Engineering**

**Prof. Arthur Goldberg**

**Dept. Genetics and Genomic Sciences**

**Spring 1, 2021**

# Class composition and inheritance in Python

## Composition

In [12]:
# composition of sexes, subjects and samples
import enum

class Sex(enum.Enum):
    # see https://docs.python.org/3/library/enum.html#using-automatic-values
    female = enum.auto()
    male = enum.auto()
    transgender_male = enum.auto()
    transgender_female = enum.auto()
    non_binary = enum.auto()
    unknown = enum.auto()

class Subject(object):
    """An individual human in a study"""
    def __init__(self, id, gender):
        self.id = id
        self.gender = gender

    def __str__(self):
        # not the use of f-string; they're very handy
        return f"id: {self.id}; gender: {self.gender.name}"

subject_1 = Subject(23, Sex.female)
print(subject_1)

id: 23; gender: female


*Now* we create `class Sample`, which reuses `Subject` via composition.

In [13]:
class Sample(object):
    """A set of subjects in a study"""
    def __init__(self, name):
        self.id = name
        self.subjects = {}

    def add(self, subject):
        # todo: check that a subject with subject.id is not already in this Sample
        self.subjects[subject.id] = subject
        
    def get(self, id):
        # todo: useful error if subject not in this Sample
        return self.subjects[id]
        
    def count(self):
        """Number of subjects"""
        return len(self.subjects)

subject_2 = Subject(78, Sex.male)
sample = Sample('example_sample')
print('sample.count():', sample.count())
sample.add(subject_1)
sample.add(subject_2)
print('sample.count():', sample.count())
print('sample.get(78)', sample.get(78))

sample.count(): 0
sample.count(): 2
sample.get(78) id: 78; gender: male


## Inheritance, and its benefits
Inheritance reuses existing class(es). It enables hierarchical refinement and extension.

The existing classes that are reused are the *base* or *superclass* classes, and the new class is the *derived* or *subclass* class. An instance of a dervied class can access the attributes (data and methods) of its base classes. It may also override them. The [`super()`](https://docs.python.org/3/library/functions.html#super) function accesses methods and attributes in the base classes.

All classes inherit from Python's built-in `object` class, the root of the class derivation DAG.

In [14]:
class Error(Exception):
    """ Base class for exceptions in this simulator package

    Attributes:
        message (:obj:`str`): the exception's message
    """
    def __init__(self, message=None):
        super().__init__(message)

# SimulatorError is derived from Error which is derived from Exception, a Python convention
class SimulatorError(Error):
    """ Exception raised for errors in this simulator package

    Attributes:
        message (:obj:`str`): the exception's message
    """
    def __init__(self, message=None):
        super().__init__(message)

TEST_MSG = 'short message'
simulator_error = SimulatorError(TEST_MSG)
print(simulator_error)

print('\nillustrate the class inheritance hierarchy')
for cls in [object, Error, SimulatorError, float]:
    print("SimulatorError is a subclass of {}: {}".format(cls.__name__,
          issubclass(SimulatorError, cls)))

print('\nillustrate the instance inheritance hierarchy')
for cls in [object, Error, SimulatorError, float]:
    print("simulator_error is an instance of {}: {}".format(cls.__name__,
          isinstance(simulator_error, cls)))

short message

illustrate the class inheritance hierarchy
SimulatorError is a subclass of object: True
SimulatorError is a subclass of Error: True
SimulatorError is a subclass of SimulatorError: True
SimulatorError is a subclass of float: False

illustrate the instance inheritance hierarchy
simulator_error is an instance of object: True
simulator_error is an instance of Error: True
simulator_error is an instance of SimulatorError: True
simulator_error is an instance of float: False


<font color='green'>Now that we have inheritance, how might we use it to define `class Sample` above?</font>

### Abstract base classes
Python [abstract base classes](https://docs.python.org/3/library/abc.html) (ABCs) are classes used to define properties shared by multiple subclasses (see [tutorial](https://pymotw.com/3/abc/)). ABCs achieve this by serving as shared nodes in the inheritance DAG:

In [15]:
from abc import ABC

class MyABC(ABC): pass

class MyClass(MyABC): pass

assert issubclass(MyClass, MyABC)
assert isinstance(MyClass(), MyABC)

# an int is not a subclass of MyABC, of course
assert isinstance(3, MyABC)

AssertionError: ignored

An ABC defines a partially enforced interface. An ABC that defines a set of methods, but not their bodies, forces subclasses to implement the methods.

ABCs cannot be instantiated themselves. Instead, they're used as superclasses of subclasses that can be instantiated.

In [16]:
from abc import ABC, abstractmethod

class C(ABC):
    # @abstractmethod decorates abstract methods
    @abstractmethod
    def my_abstract_method(self, arg1): pass

    @classmethod
    @abstractmethod
    def my_abstract_classmethod(cls): pass

    @staticmethod
    @abstractmethod
    def my_abstract_staticmethod(arg2): pass

class D(C):
    CONSTANT = 37
    def __init__(self, data):
        self.data = data
    def my_abstract_method(self, arg1):
        print(self.data, arg1)
    @classmethod
    def my_abstract_classmethod(cls):
        print('CONSTANT =', cls.CONSTANT)

    @staticmethod
    def my_abstract_staticmethod(arg3):
        print('all i know is', arg3)

d = D('my name')
d.my_abstract_method('is mud')
d.my_abstract_classmethod()
d.my_abstract_staticmethod(38)

my name is mud
CONSTANT = 37
all i know is 38


Subclasses of an ABC must implement the ABC's methods.

In [17]:
class Bad(C):

    def __init__(self, data):
        self.data = data

    def my_abstract_method(self, arg1):
        print(self.data, arg1)

    @classmethod
    def my_abstract_classmethod(cls):
        pass

Bad(1)

TypeError: ignored

But adherence to method signatures is optional.

In [18]:
class OK(C):

    def __init__(self, data):
        self.data = data

    # note the extra argument
    def my_abstract_method(self, arg1, extra_arg):
        print(self.data, arg1)

    @classmethod
    def my_abstract_classmethod(cls): pass

    @staticmethod
    def my_abstract_staticmethod(arg3): pass

ok = OK(1)
ok.my_abstract_method(2, 3)

1 2


ABCs are used by Python to define `Sequence`, `MutableSequence`, and the numeric types hierarchy (see [PEP 3141](https://www.python.org/dev/peps/pep-3141)).